# 压缩归档

如果是用于归档文件,那我们最好的选择就是使用压缩工具,python的标准库自带zipfile和tarfile用来压缩归档文件,还有gzip用来为单一文件进行压缩,另外github上还有一个归档为rar的工具[rarfile](https://github.com/markokr/rarfile)


## zipfile



zip文件格式由三个部分组成：压缩源文件数据区+压缩源文件目录区+压缩源文件目录结束标志 

> 压缩源文件数据区 

在这个数据区中每一个压缩的源文件/目录都是一条记录，记录的格式如下： [文件头+ 文件数据 + 数据描述符]
    
+ 文件头结构 

组成| 长度 
---|---
文件头标记| 4 bytes (0x04034b50) 
解压文件所需 pkware 版本 |2 bytes 
全局方式位标记 |2 bytes 
压缩方式 |2 bytes 
最后修改文件时间 |2 bytes 
最后修改文件日期 |2 bytes 
CRC-32校验 |4 bytes 
压缩后尺寸 |4 bytes 
未压缩尺寸 |4 bytes 
文件名长度 |2 bytes 
扩展记录长度 |2 bytes 
文件名 |（不定长度） 
扩展字段 |（不定长度）
       
+ 文件数据 
    
+ 数据描述符 
    
组成 |长度 
---|---
CRC-32校验 |4 bytes 
压缩后尺寸 |4 bytes 
未压缩尺寸 |4 bytes 

这个数据描述符只在全局方式位标记的第３位设为１时才存在，紧接在压缩数据的最后一个字节后。这个数据描述符只用在不能对输出的 ZIP 文件进行检索时使用。例如：在一个不能检索的驱动器（如：磁带机上）上的 ZIP 文件中。如果是磁盘上的ZIP文件一般没有这个数据描述符。 


> 压缩源文件目录区 

在这个数据区中每一条纪录对应在压缩源文件数据区中的一条数据 

 组成 |长度 
 ---|---
目录中文件文件头标记| 4 bytes (0x02014b50) 
压缩使用的pkware 版本| 2 bytes 
解压文件所需 pkware 版本| 2 bytes 
全局方式位标记 |2 bytes 
压缩方式| 2 bytes 
最后修改文件时间| 2 bytes 
最后修改文件日期 |2 bytes 
ＣＲＣ－３２校验 |4 bytes 
压缩后尺寸| 4 bytes 
未压缩尺寸| 4 bytes 
文件名长度 |2 bytes 
扩展字段长度| 2 bytes 
文件注释长度 |2 bytes 
磁盘开始号 |2 bytes 
内部文件属性 |2 bytes 
外部文件属性 |4 bytes 
局部头部偏移量 |4 bytes 
文件名 |（不定长度） 
扩展字段 |（不定长度） 
文件注释 |（不定长度） 


> 压缩源文件目录结束标志 

组成 |长度 
---|---
目录结束标记 |4 bytes (0x02014b50) 
当前磁盘编号 |2 bytes 
目录区开始磁盘编号| 2 bytes 
本磁盘上纪录总数 |2 bytes 
目录区中纪录总数| 2 bytes 
目录区尺寸大小 |4 bytes 
目录区对第一张磁盘的偏移量| 4 bytes 
ZIP 文件注释长度 |2 bytes 
ZIP 文件注释 |（不定长度）

In [1]:
import zipfile

### 创建归档

+ #### 创建一个文件的zip归档

In [2]:
with zipfile.ZipFile('source/output/笑傲江湖.zip', 'w',zipfile.ZIP_DEFLATED) as f:
    f.write("source/input/笑傲江湖.txt")

+ #### 创建字符串的归档

In [3]:
with open("source/input/iris.csv","r") as f:
    content = f.read()

In [4]:
with zipfile.ZipFile('source/output/iris_str.zip', 'w',zipfile.ZIP_DEFLATED) as f:
    f.writestr( 'iris_str.csv',content)

+ #### 创建一个多文件归档

在归档大量文件时,我们可以用`allowZip64=True`来指定支持超过2Gb的归档

In [5]:
with zipfile.ZipFile('source/output/all_input.zip', 'w',zipfile.ZIP_DEFLATED,allowZip64=True) as f:
    f.write("source/input/笑傲江湖.txt")
    f.write("source/input/iris.csv")
    f.write("source/input/people.json")

### 查看压缩文件信息

In [6]:
#查看是不是zip压缩文件
zipfile.is_zipfile("source/output/all_input.zip") 

True

In [7]:
# 查看zip中的文件列表
with zipfile.ZipFile('source/output/all_input.zip', 'r',zipfile.ZIP_DEFLATED) as f:
    print(f.namelist())

['source/input/笑傲江湖.txt', 'source/input/iris.csv', 'source/input/people.json']


In [8]:
# 打开zip中某个文件
with zipfile.ZipFile('source/output/all_input.zip', 'r',zipfile.ZIP_DEFLATED) as f:
    print(f.open('source/input/people.json').read().decode("utf-8"))

[{"name":"Michael"},{"name":"Andy", "age":30},{"name":"Justin", "age":19}]



In [9]:
# 查看zip文件的信息列表
with zipfile.ZipFile('source/output/all_input.zip', 'r',zipfile.ZIP_DEFLATED) as f:
    print(f.infolist())

[<ZipInfo filename='source/input/笑傲江湖.txt' compress_type=deflate filemode='-rwxr-xr-x' file_size=2989594 compress_size=1269072>, <ZipInfo filename='source/input/iris.csv' compress_type=deflate filemode='-rwxr-xr-x' file_size=4606 compress_size=873>, <ZipInfo filename='source/input/people.json' compress_type=deflate filemode='-rwxr-xr-x' file_size=75 compress_size=52>]


In [10]:
# 查看压缩信息
with zipfile.ZipFile('source/output/all_input.zip', 'r',zipfile.ZIP_DEFLATED) as f:
    print(f.printdir())

File Name                                             Modified             Size
source/input/笑傲江湖.txt                          2016-12-23 23:17:44      2989594
source/input/iris.csv                          2016-12-23 23:17:44         4606
source/input/people.json                       2016-12-23 23:17:44           75
None


In [11]:
# 查看zip中某文件的信息
with zipfile.ZipFile('source/output/all_input.zip', 'r',zipfile.ZIP_DEFLATED) as f:
    info = f.getinfo('source/input/people.json')
    print(info)

<ZipInfo filename='source/input/people.json' compress_type=deflate filemode='-rwxr-xr-x' file_size=75 compress_size=52>


In [12]:
# 查看创建时间

with zipfile.ZipFile('source/output/all_input.zip', 'r',zipfile.ZIP_DEFLATED) as f:
    info = f.getinfo('source/input/people.json')
    print(info.date_time)

(2016, 12, 23, 23, 17, 44)


In [13]:
# 检查zip中每个文件的CRC,有错误会返回对应文件作为列表成员
with zipfile.ZipFile('source/output/all_input.zip', 'r',zipfile.ZIP_DEFLATED) as f:
    print(f.testzip())

None


### 解压文件

+ #### 全部解压

In [14]:
with zipfile.ZipFile('source/output/all_input.zip', 'r',zipfile.ZIP_DEFLATED) as f:
    f.extractall("source/extract")

+ #### 单独解压一个

In [15]:
with zipfile.ZipFile('source/output/all_input.zip', 'r',zipfile.ZIP_DEFLATED) as f:
    f.extract("source/input/iris.csv","source/exone")

### 密码处理

zipfile只能解压带密码的zip包,并不支持创建加密的zip归档,要使用密码只要像这个样:

```python

with zipfile.ZipFile('source/output/all_input.zip', 'r',zipfile.ZIP_DEFLATED) as f:
    f.setpassword(<pwd>)
    f.extract("source/input/iris.csv","source/exone")
```

即可

## tarfile

tar是linux下是常见的归档格式,常见的后缀有tar,tar.bz,tar.gz三种后缀,分别对应三种不同的压缩算法,

tarfile的归档用法也与zipfile类似,只是接口有些变化


In [16]:
import tarfile

In [17]:
with tarfile.TarFile('source/output/all_input.tar', 'w') as f:
    f.add("source/input/笑傲江湖.txt")
    f.add("source/input/iris.csv")
    f.add("source/input/people.json")

In [18]:
#查看是不是zip压缩文件
tarfile.is_tarfile("source/output/all_input.tar") 

True

In [19]:
#查看压缩信息
with tarfile.TarFile('source/output/all_input.tar', 'r') as f:
    print(f.list())

?rwxr-xr-x huangsizhe/staff    2989594 2016-12-23 23:17:44 source/input/笑傲江湖.txt 
?rwxr-xr-x huangsizhe/staff       4606 2016-12-23 23:17:44 source/input/iris.csv 
?rwxr-xr-x huangsizhe/staff         75 2016-12-23 23:17:44 source/input/people.json 
None


In [20]:
# 查看zip中的文件列表
with tarfile.TarFile('source/output/all_input.tar', 'r') as f:
    print(f.getnames())

['source/input/笑傲江湖.txt', 'source/input/iris.csv', 'source/input/people.json']


In [21]:
# 查看zip文件的信息列表
with tarfile.TarFile('source/output/all_input.tar', 'r') as f:
    print(f.getmembers())

[<TarInfo 'source/input/笑傲江湖.txt' at 0x10e66db38>, <TarInfo 'source/input/iris.csv' at 0x10e66da70>, <TarInfo 'source/input/people.json' at 0x10e66de58>]


In [22]:
# 查看zip中某文件的信息,比如修改时间戳
with tarfile.TarFile('source/output/all_input.tar', 'r') as f:
    print(f.getmember("source/input/iris.csv").mtime)

1482506264


如果要结合gz或者bz压缩,那么就不能使用这个类,而要使用`tarfile.open(name=None, mode='r', fileobj=None, bufsize=10240, **kwargs)`函数

其中mode=可选的有:

mode|说明
---|---
`'r' or 'r:*'`|使用透明压缩读打开
`'r:'`|无压缩读打开
`'r:gz'`|gzip压缩读打开
`'r:bz2'`|bzip2压缩读打开
`'a' or 'a:'`|无需压缩append写打开。如果文件不存在，则创建该文件。
`'w' or 'w:'`|无压缩写
`'w:gz'`|gzip写打开
`'w:bz2'`|bzip2写打开

+ #### 创建压缩归档

In [23]:
import os
with tarfile.open("source/output/all_input.tar.gz","w:gz") as tar:
    for root,dir,files in os.walk("source/input"):
        for file in files:
            fullpath = os.path.join(root,file)
            tar.add(fullpath)



+ #### 解压压缩归档

In [24]:
with tarfile.open("source/output/all_input.tar.gz","r:gz") as tar:
    names = tar.getnames()
    for name in names:
        tar.extract(name,path="source/ex_tar")

## [rarfile](https://github.com/markokr/rarfile)

python标准库并不支持rar格式的归档,但有个rarfile可以通过pip安装,他的接口与zipfile一样,只是不能写只能读

